# Organoid Batch Processing Workflow

This notebook runs the workflow demonstrated in [Single_image_workflow.ipynb](Single_image_workflow.ipynb) for all images in a specified folder.

To run this notebook, you need to place all organoid images in a folder named `Organoids` under a `data` folder in the root of this repository (or adjust the path accordingly).

Below is an exampe of the folder structure:

```
Brightfield-3D-models-image-analysis
|
├─ code
|   |
|   ⁞
|   
├─ data
|   |
⁞   ├─ Organoids
    |    |
    ⁞    ├─ Org43_1.tiff
         ├─ Org43_2.tiff
         ├─ Org43_3.tiff
         ⁞
```

The conda environment used for this notebook can be created following the instructions in the [README.md](README.md) file. But in short, we need an environment with the [devbio-napari plugin](https://github.com/haesleinhuepf/devbio-napari).

## Import libraries

In [1]:
from skimage.io import imread, imsave
from pathlib import Path
import napari_simpleitk_image_processing as nsitk
import pyclesperanto_prototype as cle
from napari_skimage_regionprops import regionprops_table, add_table, get_table

## Function with the workflow

Basically, this was created by copy-pasting, in the same order, each command from the workflow notebook.

Then, a generic input argument `image_path` was provided and 2 return variables were added: `image8_eloe` (the final labeled image) and `table`.

If you want to measure more things than just size, take a look at the arguments of the function `regionprops_table` and change them from `False` to `True`. For example, if you want shape measurements, provide an extra argument `shape=True,`.

In [2]:
def workflow(image_path):

    image0_O1 = imread(image_path)
    
    image1_L = nsitk.laplacian_of_gaussian_filter(image0_O1, 1.0)
    
    image2_S = nsitk.standard_deviation_filter(image1_L, 1, 1, 0)
    
    image3_to = cle.threshold_otsu(image2_S)
    
    image4_cl = cle.closing_labels(image3_to, None, 1.0)
    
    image5_B = nsitk.binary_fill_holes(image4_cl)
    
    image6_C = cle.label(image5_B)
    
    image7_esl = cle.exclude_small_labels(image6_C, None, 300.0)
    
    image8_eloe = cle.exclude_labels_on_edges(image7_esl)
    
    table = regionprops_table(
        image0_O1,
        image8_eloe,
        size=True,
        intensity=False,
        napari_viewer=None,
    )
    
    return image8_eloe, table

## Path to the images

Please replace the path below to your local data folder.

In [3]:
data_path =  Path("../../data/Organoids/")

Store image paths to a list of paths.

In [4]:
# Create empty list of paths
image_path_list = []
# iterate over folder path
for path in data_path.iterdir():
    # if path ends with ".tiff", it is a path to an image
    if (path.suffix == '.tiff'):
        # Append the path to the list
        image_path_list += [path]

Below we display the list of paths.

In [5]:
image_path_list

[WindowsPath('../../data/Organoids/Org43_1.tiff'),
 WindowsPath('../../data/Organoids/Org43_2.tiff'),
 WindowsPath('../../data/Organoids/Org43_3.tiff')]

## Batch Processing

For each path in the list, read the image, apply the workflow and save the results in an "Outputs" folder.

Labeled images and tables are saved inside the Outputs folder with the same name as the original image + "\_labels" or "\_table", respectively. 

In [6]:
# iterate over list of paths
for image_path in image_path_list:
    # applies workflow on image
    label_image, table = workflow(image_path)
    
    # Create an empty "Outputs" folder (if not there already)
    output_folder_path = Path(image_path.parent, 'Outputs')
    output_folder_path.mkdir(exist_ok = True)
    
    # Create a "image_labels.tif" path
    output_label_image_path = Path(output_folder_path, image_path.stem + '_labels.tif')
    # Store the label_image in the created "image_labels.tif" path
    imsave(output_label_image_path, label_image)
    
    # Create a "image_table.csv" path
    output_table_path = Path(output_folder_path, image_path.stem + '_table.csv')
    # Store the table in the created "image_table.csv" path
    table.to_csv(output_table_path)

C:\Users\mazo260d\AppData\Local\Temp\ipykernel_16072\1802166967.py:13: UserWarning: C:\Users\mazo260d\Documents\GitHub\Brightfield-3D-models-image-analysis\data\Organoids\Outputs\Org43_1_labels.tif is a low contrast image
  imsave(output_label_image_path, label_image)
C:\Users\mazo260d\AppData\Local\Temp\ipykernel_16072\1802166967.py:13: UserWarning: C:\Users\mazo260d\Documents\GitHub\Brightfield-3D-models-image-analysis\data\Organoids\Outputs\Org43_2_labels.tif is a low contrast image
  imsave(output_label_image_path, label_image)
C:\Users\mazo260d\AppData\Local\Temp\ipykernel_16072\1802166967.py:13: UserWarning: C:\Users\mazo260d\Documents\GitHub\Brightfield-3D-models-image-analysis\data\Organoids\Outputs\Org43_3_labels.tif is a low contrast image
  imsave(output_label_image_path, label_image)
